### Lấy dự liệu và set-up

In [ ]:
import os

# Đường dẫn đến thư mục ảnh và nhãn
image_path = r'data/train/images'
label_path = r'data/train/labels'
output_file = 'dataset.txt'

# Mở tệp đầu ra để ghi dữ liệu tổng hợp
with open(output_file, 'w') as outfile:
    # Duyệt qua tất cả các file trong thư mục nhãn
    for label_file in os.listdir(label_path):
        if label_file.endswith('.txt'):
            # Tạo đường dẫn đến tệp nhãn và lấy tên file ảnh
            label_file_path = os.path.join(label_path, label_file)
            image_file_name = label_file.replace('.txt', '')  # Chỉ lấy tên file ảnh, bỏ phần path và .txt

            # Đọc nội dung của file nhãn và ghi vào file tổng hợp
            with open(label_file_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    # Thêm tên file ảnh vào trước dòng nhãn và ghi vào file đầu ra
                    outfile.write(f"{image_file_name} {line.strip()}\n")


### Eda data cho dataframe



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def process_dataframe(file_path):
    df = pd.read_csv(file_path, sep=' ', header=None)

    if df.shape[1] == 6:
        df.columns = ['image_name', 'class_id', 'x_center', 'y_center', 'width', 'height']
    elif df.shape[1] == 7:
        df.columns = ['image_name', 'class_id', 'x_center', 'y_center', 'width', 'height', 'confidence_score']
    else:
        raise ValueError("Số lượng cột không hợp lệ trong file.")

    def extract_cam_name(image_name):
        if image_name.startswith("cam_") or image_name.startswith("src_"):
            parts = image_name.split('_')
            if len(parts) >= 2:
                return f"{parts[0]}_{int(parts[1])}"  
            else:
                raise ValueError("Tên ảnh không hợp lệ, không đủ phần.")
        else:
            raise ValueError("Tên ảnh không hợp lệ, cần bắt đầu bằng 'cam_' hoặc 'src_'.")

    df['cam_name'] = df['image_name'].apply(extract_cam_name)

    return df

In [ ]:
df = process_dataframe('dataset.txt')

In [ ]:
df.describe()

In [ ]:
class_mapping = {
    0: 'motor',
    1: 'car',
    2: 'bus',
    3: 'truck'
}

# 2. Phân bố số lượng các class_id
plt.figure(figsize=(10, 6))
sns.countplot(x='class_id', data=df, palette='viridis')
plt.title("Phân bố số lượng các class_id")
plt.xlabel("Class ID")
plt.ylabel("Số lượng")
plt.xticks(ticks=range(len(class_mapping)), labels=list(class_mapping.values()))
plt.show()

# 3. Kiểm tra kích thước của bounding boxes
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['width'], bins=30, kde=True, color='skyblue')
plt.title("Phân phối của Width (Chiều rộng)")
plt.xlabel("Width")
plt.ylabel("Số lượng")

plt.subplot(1, 2, 2)
sns.histplot(df['height'], bins=30, kde=True, color='salmon')
plt.title("Phân phối của Height (Chiều cao)")
plt.xlabel("Height")
plt.ylabel("Số lượng")

plt.tight_layout()
plt.show()

# 4. Phân tích tọa độ trung tâm
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['x_center'], bins=30, kde=True, color='olive')
plt.title("Phân phối của X Center (Tọa độ trung tâm X)")
plt.xlabel("X Center")
plt.ylabel("Số lượng")

plt.subplot(1, 2, 2)
sns.histplot(df['y_center'], bins=30, kde=True, color='purple')
plt.title("Phân phối của Y Center (Tọa độ trung tâm Y)")
plt.xlabel("Y Center")
plt.ylabel("Số lượng")

plt.tight_layout()
plt.show()



In [ ]:
class_counts = df['class_id'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.title('Tỉ lệ phần trăm các class_id', fontsize=15)
plt.axis('equal')  # Để biểu đồ tròn trông cân đối
plt.show()
# 3. Phân tích kích thước của bounding boxes theo class_id
plt.figure(figsize=(12, 6))
sns.boxplot(x='class_id', y='x_center', data=df, palette='Blues')
plt.title("Phân phối x của bounding box theo class_id")
plt.xlabel("Class ID")
plt.xticks(ticks=range(len(class_mapping)), labels=list(class_mapping.values()))  # Thay đổi nhãn trục X

plt.ylabel("Chiều rộng (Width)")
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='class_id', y='y_center', data=df, palette='Oranges')
plt.title("Phân phối y của bounding box theo class_id")
plt.xlabel("Class ID")
plt.xticks(ticks=range(len(class_mapping)), labels=list(class_mapping.values()))  # Thay đổi nhãn trục X
plt.ylabel("Chiều cao (Height)")
plt.show()

# 3. Phân tích kích thước của bounding boxes theo class_id
plt.figure(figsize=(12, 6))
sns.boxplot(x='class_id', y='width', data=df, palette='Blues')
plt.title("Phân phối chiều rộng của bounding box theo class_id")
plt.xlabel("Class ID")
plt.xticks(ticks=range(len(class_mapping)), labels=list(class_mapping.values()))  # Thay đổi nhãn trục X
plt.ylabel("Chiều rộng (Width)")
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='class_id', y='height', data=df, palette='Oranges')
plt.title("Phân phối chiều cao của bounding box theo class_id")
plt.xlabel("Class ID")
plt.xticks(ticks=range(len(class_mapping)), labels=list(class_mapping.values()))  # Thay đổi nhãn trục X
plt.ylabel("Chiều cao (Height)")
plt.show()


# 4. Phân tích vị trí trung tâm của bounding boxes
plt.figure(figsize=(12, 6))
sns.jointplot(x='x_center', y='y_center', data=df, kind="scatter", alpha=0.5, color='purple')
plt.suptitle("Phân phối vị trí trung tâm của bounding boxes")
plt.xlabel("X Center")
plt.ylabel("Y Center")
plt.tight_layout()
plt.show()


# Insight 4: Kiểm tra sự phân bố của tọa độ trung tâm theo class_id
plt.figure(figsize=(14, 7))
sns.scatterplot(x='x_center', y='y_center', hue='class_id', data=df, palette='husl', alpha=0.6)
plt.title("Vị trí trung tâm của bounding boxes theo class_id")
plt.xlabel("X Center")
plt.ylabel("Y Center")
plt.legend(title="Class ID")
plt.show()

# Insight 5: Tương quan giữa kích thước (width, height) và vị trí trung tâm
plt.figure(figsize=(14, 7))
sns.scatterplot(x='width', y='height', hue='class_id', data=df, palette='coolwarm', alpha=0.6)
plt.title("Mối quan hệ giữa chiều rộng và chiều cao của bounding box theo class_id")
plt.xlabel("Width")
plt.ylabel("Height")
plt.legend(title="Class ID")
plt.show()

# Insight 1: Tìm hiểu kích thước trung bình và phân phối
average_width = df['width'].mean()
average_height = df['height'].mean()
print(f"\nChiều rộng trung bình: {average_width:.2f}, Chiều cao trung bình: {average_height:.2f}")

# 5. Phân tích số lượng bounding boxes theo từng ảnh
boxes_per_image = df['image_name'].value_counts()
plt.figure(figsize=(10, 6))
sns.histplot(boxes_per_image, bins=30, kde=True, color='purple')
plt.title("Phân bố số lượng bounding boxes theo từng ảnh")
plt.xlabel("Số lượng bounding boxes")
plt.ylabel("Tần suất")
plt.show()

# Insight 2: Số lượng ảnh có nhiều bounding boxes
images_with_multiple_boxes = boxes_per_image[boxes_per_image > 1].count()
print(f"\nSố lượng ảnh có nhiều hơn 1 bounding box: {images_with_multiple_boxes}")

# Giả sử kích thước ảnh cố định
image_width = 1280
image_height = 720

# Chuyển đổi từ tọa độ tương đối sang tọa độ pixel
x_coords = (df["x_center"] * image_width).astype(int)
y_coords = (df["y_center"] * image_height).astype(int)

# Tạo 2D histogram
plt.hist2d(x_coords, y_coords, bins=[50, 50], cmap="viridis")

# Thêm tiêu đề và nhãn trục
plt.title("2D Histogram of Bounding Box Center Coordinates")
plt.xlabel("Center X-coordinate")
plt.ylabel("Center Y-coordinate")

# Hiển thị thanh màu
plt.colorbar(label="Count")

# Hiển thị biểu đồ
plt.show()


In [ ]:
df['area'] = df['width'] * df['height']

# Giả sử kích thước ảnh đã biết
image_height = 720  # Chiều cao ảnh
image_width = 1280   # Chiều rộng ảnh

df['img_width'] = image_width
df['img_height'] = image_height

# Tính toán tọa độ (xmin, ymin, xmax, ymax)
df['xmin'] = df['x_center'] - (df['width'] / 2)
df['ymin'] = df['y_center'] - (df['height'] / 2)
df['xmax'] = df['x_center'] + (df['width'] / 2)
df['ymax'] = df['y_center'] + (df['height'] / 2)

# Tính diện tích bounding box
df['area'] = (df['xmax'] - df['xmin']) * (df['ymax'] - df['ymin'])

# Tính tỷ lệ diện tích
df['ratio'] = df['area'] / (df['img_width'] * df['img_height'])

# Ánh xạ class_id sang tên lớp
df['class_name'] = df['class_id'].map(class_mapping)

# 2. Vẽ biểu đồ boxplot cho tỷ lệ diện tích
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['ratio'])
plt.xlabel('Tỷ lệ diện tích')
plt.title('Boxplot cho Tỷ lệ Diện tích Bounding Box')
plt.grid()
plt.show()

# 3. Vẽ biểu đồ phân phối Density cho tỷ lệ diện tích
plt.figure(figsize=(10, 6))
sns.kdeplot(df['ratio'], bw_adjust=0.5)
plt.xlabel('Tỷ lệ diện tích')
plt.title('Phân phối Density của Tỷ lệ Diện tích Bounding Box')
plt.grid()
plt.show()

# 4. Phân tích tỷ lệ diện tích theo class_name
plt.figure(figsize=(12, 8))
sns.boxplot(x='class_name', y='ratio', data=df)
plt.xlabel('Class Name')
plt.ylabel('Tỷ lệ diện tích')
plt.title('Tỷ lệ Diện tích của Bounding Box theo Class Name')
plt.grid()
plt.show()

# 7. Tổng hợp tỷ lệ diện tích theo từng class_name
class_area_ratio = df.groupby('class_name')['ratio'].mean().reset_index()
class_area_ratio.columns = ['class_name', 'mean_area_ratio']

plt.figure(figsize=(10, 6))
sns.barplot(x='class_name', y='mean_area_ratio', data=class_area_ratio)
plt.title('Tỷ lệ Diện tích Trung bình theo Class Name')
plt.xlabel('Class Name')
plt.ylabel('Tỷ lệ Diện tích Trung bình')
plt.grid()
plt.show()

# 8. Phân phối diện tích
plt.figure(figsize=(10, 6))
sns.histplot(df['area'], bins=30, kde=True, color='skyblue')
plt.title("Phân phối của Area")
plt.xlabel("Area")
plt.ylabel("Tần suất")
plt.show()

# 9. Tính tọa độ pixel và vẽ biểu đồ phân tán
df["x_pixel"] = df["x_center"] * image_width
df["y_pixel"] = df["y_center"] * image_height

plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['x_pixel'], df['y_pixel'], c=df['area'], cmap='viridis', s=100, alpha=0.7, edgecolors='k')
cbar = plt.colorbar(scatter)
cbar.set_label('Area (pixels^2)', fontsize=12)
cbar.ax.tick_params(labelsize=10)
plt.xlabel('X (pixels)', fontsize=12)
plt.ylabel('Y (pixels)', fontsize=12)
plt.title("Mối quan hệ giữa X, Y và Area", fontsize=14)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

# 10. Phân phối diện tích theo Class Name
plt.figure(figsize=(12, 6))
sns.boxplot(x='class_name', y='area', data=df)
plt.title("Phân phối của Area theo Class Name")
plt.xlabel("Class Name")
plt.ylabel("Area")
plt.show()

### Visualine bouding box

In [ ]:
from typing import List
import random
import cv2

In [ ]:

color_mapping = {
    0: (0, 255, 0),    # Xanh lá cây cho motor
    1: (0, 0, 255),    # Đỏ cho car
    2: (255, 0, 0),    # Xanh dương cho bus
    3: (255, 255, 0)   # Vàng cho truck
}

def draw_all_bounding_boxes(image, data_rows):
    h, w = image.shape[:2]
    for data_row in data_rows:
        xmin = int((data_row['x_center'] - data_row['width'] / 2) * w)
        ymin = int((data_row['y_center'] - data_row['height'] / 2) * h)
        xmax = int((data_row['x_center'] + data_row['width'] / 2) * w)
        ymax = int((data_row['y_center'] + data_row['height'] / 2) * h)

        label = class_mapping[data_row['class_id']]
        color = color_mapping[data_row['class_id']]

        # Vẽ bounding box
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 5)

        # Vẽ nhãn
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        cv2.rectangle(image, (xmin, ymin - text_size[1] - 10), (xmin + text_size[0], ymin - 10), color, -1)
        cv2.putText(image, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return image

def display_images_with_bboxes(target_image_names, df):
    if not isinstance(target_image_names, list):
        print("target_image_names phải là một danh sách.")
        return

    for target_image_name in target_image_names:
        filtered_df = df[df['image_name'] == target_image_name]
        if filtered_df.empty:
            print(f"Không có dữ liệu cho ảnh: {target_image_name}.jpg")
            continue
        image_path = os.path.join(image_directory, target_image_name)
        if not image_path.endswith('.jpg'):
            image_path = f"{image_path}.jpg"
        image = cv2.imread(image_path)

        if image is not None:
            image_with_boxes = draw_all_bounding_boxes(image.copy(), filtered_df.to_dict(orient='records'))

            plt.figure(figsize=(10, 6))
            plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
            plt.title(f"Objects in: {target_image_name}.jpg", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            print(f"Không tìm thấy ảnh: {target_image_name}.jpg")


def get_random_images_from_cam(df: pd.DataFrame, cam_names: List[str] = None, num_images: int = 50) -> List[str]:
    if cam_names is None:
        all_cams = df['image_name'].str.extract(r'(cam_\d+)')[0].unique().tolist()
        cam_names = all_cams

    filtered_df = df[df['image_name'].str.extract(r'(cam_\d+)')[0].isin(cam_names)]
    unique_images = filtered_df['image_name'].unique()
    random_images = random.sample(list(unique_images), num_images)

    return random_images


In [ ]:
#Ảnh boudingbox tập train
df = process_dataframe('dataset.txt')
image_directory = r"data/train/images"
random_images = get_random_images_from_cam(df)
target_image_names = random_images
display_images_with_bboxes(target_image_names, df)